In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

hexnut=cv.imread('hexnut_template.png',cv.IMREAD_COLOR)
squarenut=cv.imread('squarenut_template.png',cv.IMREAD_COLOR)
conv=cv.imread('conveyor_f100.png',cv.IMREAD_COLOR)

assert hexnut is not None
assert squarenut is not None
assert conv is not None

fig,ax=plt.subplots(1,3,figsize=(12,36))
for i in range(3):
    ax[i].set_axis_off()
ax[0].imshow(cv.cvtColor(hexnut,cv.COLOR_BGR2RGB))
ax[1].imshow(cv.cvtColor(squarenut,cv.COLOR_BGR2RGB))
ax[2].imshow(cv.cvtColor(conv,cv.COLOR_BGR2RGB))

Thresholding

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

hexnut=cv.imread('hexnut_template.png',cv.IMREAD_GRAYSCALE)
squarenut=cv.imread('squarenut_template.png',cv.IMREAD_GRAYSCALE)
conv=cv.imread('conveyor_f100.png',cv.IMREAD_GRAYSCALE)

assert hexnut is not None
assert squarenut is not None
assert conv is not None

rethex,threshex=cv.threshold(hexnut,0,255,cv.THRESH_OTSU)
retsq,thressq=cv.threshold(squarenut,0,255,cv.THRESH_OTSU)
retconv,thresconv=cv.threshold(conv,0,255,cv.THRESH_OTSU)

print('Threshold value of hexnut is : ',rethex)
print('Threshold value of squarenut is : ',retsq)
print('Threshold value of conveyor is : ',retconv)

fig,ax=plt.subplots(1,3,figsize=(12,36))
for i in range(3):
    ax[i].set_axis_off()
ax[0].imshow(threshex,cmap='gray')
ax[1].imshow(thressq,cmap='gray')
ax[2].imshow(thresconv,cmap='gray')


In here, background is white while foreground is black. Therefore, inverse transformation is used to change the colors.

In [ ]:
t=255-np.arange(0,256,1).astype(np.uint8)
assert t[0]==255 #Length of array is 256

threshex=t[threshex] #Indexing the transformation t with the values of pixels in the img
thressq=t[thressq]
thresconv=t[thresconv]

fig,ax=plt.subplots(1,3,figsize=(12,36))
for i in range(3):
    ax[i].set_axis_off()
ax[0].imshow(threshex,cmap='gray')
ax[1].imshow(thressq,cmap='gray')
ax[2].imshow(thresconv,cmap='gray')

Morphological closing

In [ ]:
se=np.ones((3,3),np.uint8)

threshex2=cv.morphologyEx(threshex,cv.MORPH_CLOSE,se)
thressq2=cv.morphologyEx(thressq,cv.MORPH_CLOSE,se)
thresconv2=cv.morphologyEx(thresconv,cv.MORPH_CLOSE,se)

fig,ax=plt.subplots(1,6,figsize=(12,36))
for i in range(6):
    ax[i].set_axis_off()
ax[0].imshow(threshex,cmap='gray')
ax[0].set_title('original')
ax[1].imshow(threshex2,cmap='gray')
ax[1].set_title('Closed')
ax[2].imshow(thressq,cmap='gray')
ax[2].set_title('original')
ax[3].imshow(thressq2,cmap='gray')
ax[3].set_title('Closed')
ax[4].imshow(thresconv,cmap='gray')
ax[4].set_title('original')
ax[5].imshow(thresconv2,cmap='gray')
ax[5].set_title('Closed')

Connected component analysis

In [ ]:
rethex,labelhex,statshex,centroidhex=cv.connectedComponentsWithStats(threshex2)
retsq,labelsq,statssq,centroidsq=cv.connectedComponentsWithStats(thressq2)
retconv,labelconv,statsconv,centroidconv=cv.connectedComponentsWithStats(thresconv2)

labelhex=cv.normalize(labelhex,0,255,cv.NORM_MINMAX).astype(np.uint8)
labelsq=cv.normalize(labelsq,0,255,cv.NORM_MINMAX).astype(np.uint8)
labelconv=cv.normalize(labelconv,0,255,cv.NORM_MINMAX).astype(np.uint8)

indexhex=cv.applyColorMap(labelhex,cv.COLORMAP_JET)
indexsq=cv.applyColorMap(labelsq,cv.COLORMAP_JET)
indexconv=cv.applyColorMap(labelconv,cv.COLORMAP_JET)

fig,ax=plt.subplots(1,3,figsize=(16,48))
ax[0].imshow(labelhex)
ax[0].set_title('Hexnut')
ax[1].imshow(labelsq)
ax[1].set_title('Squarenut')
ax[2].imshow(labelconv)
ax[2].set_title('Conveyor belt')

In [ ]:
print('Connected component in Hexnut image is : ',rethex)
print('Connected component in Squarenut image is : ',retsq)
print('Connected component in Conveyor_f100 image is : ',retconv)

print('\n')
print('Hexnut - location = ',statshex[1][0],statshex[1][1])
print('Hexnut - Width = ',statshex[1][2])
print('Hexnut - Height = ',statshex[1][3])
print('Hexnut - Area = ',statshex[1][4])
print('Hexnut - Centroid = ',centroidhex[1])

print('\n')
print('Squarenut - location = ',statssq[1][0],statssq[1][1])
print('squarenut - Width = ',statssq[1][2])
print('squareut - Height = ',statssq[1][3])
print('Squarenut - Area = ',statssq[1][4])
print('Squarenut - Centroid = ',centroidsq[1])

print('\n')
for i in range(1, retconv):
    print('Component', i)
    print('    Location:', statsconv[i][0], statsconv[i][1])
    print('    Width:', statsconv[i][2])
    print('    Height:', statsconv[i][3])
    print('    Area:', statsconv[i][4])
    print('    Centroid:', centroidconv[i])
    print('\n')


Converting label image into a binary image

In [ ]:
binary = np.zeros_like(labelconv, dtype=np.uint8)
for label in range(1, labelconv.max() + 1):
    binary[labelconv == label] = 1

# Return the resulting binary image
binary_image = binary.astype(bool)*255
binary_image=binary_image.astype(np.uint8)
plt.imshow(binary_image,cmap='gray')

In [ ]:
contours, hierarchy = cv.findContours(binary_image, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
bin=binary_image.copy()

img = cv.drawContours(bin, contours, -1, (150, 150, 250), 8)
fig,ax=plt.subplots(1,2,figsize=(16,48))
ax[0].imshow(img)
ax[0].set_title('With contours')
ax[1].imshow(binary_image)
ax[1].set_title('original')

In [ ]:
l=[]
for i in range(len(contours)):
    epsilon = 0.01 * cv.arcLength(contours[i], True)
    approx_polygon = cv.approxPolyDP(contours[i], epsilon, True)
    l.append(len(approx_polygon))
print('Number of Hexnuts : ',l.count(6))
print('Number of Squarenuts : ',l.count(4))



Another method using cv.matchshapes()

In [ ]:
#Here contour[3] is for the hexogonal nut
same=[]
diff=[]
for i in range(4):
    for j in range(4):
        if i!=j:
            match_value = cv.matchShapes(contours[i], contours[j], 1, 0.0)
            if match_value>0.001: #this value was used as the threshold
                if [j,i] not in diff:
                    diff.append([i,j])
                
            else:
                if [j,i] not in same:
                    same.append([i,j])


In [ ]:
s=[]
for i in same:
    for j in i:
        s.append(j)
s=list(set(s))

if 3 in s:
    n_hex=len(s)
    n_sq=len(contours)-len(s)
else:
    n_sq=len(s)
    n_hex=len(contours)-len(s)

print('Number of Hexnuts : ',n_hex)
print('Number of Squarenuts : ',n_sq)

Detecting Objects on a Synthetic Conveyor

In [ ]:
import cv2 as cv
import numpy as np
from scipy.optimize import linear_sum_assignment

# Initialize the list of objects
objects = []
ll=[]
frame_array = []

fn = 'conveyor.mp4'
cap = cv.VideoCapture(fn)

if not cap.isOpened():
    print('Cannot open the file {}'.format(fn))
cv.namedWindow('Conveyor',  cv.WINDOW_FULLSCREEN)

f = 0
frame = [] 
a=[]     

while cap.isOpened():                                    
    ret, frame = cap.read()
    if not ret:
        print("Can't receive frame (stream end?). Exiting.")
        break
    
    gray=cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
    ret,thresh=cv.threshold(gray,0,255,cv.THRESH_OTSU)

    t=255-np.arange(0,256,1).astype(np.uint8)
    thresh=t[thresh] 
    
    se=np.ones((3,3),np.uint8)
    thresh=cv.morphologyEx(thresh,cv.MORPH_CLOSE,se)
    num_labels, labels, stats, centroids = cv.connectedComponentsWithStats(thresh)
    labels=cv.normalize(labels,0,255,cv.NORM_MINMAX).astype(np.uint8)

    binary = np.zeros_like(labels, dtype=np.uint8)
    for label in range(1, labels.max() + 1):
        binary[labels == label] = 1

    # Return the resulting binary image
    binary_image = binary.astype(bool)*255
    labelss=binary_image.astype(np.uint8)
    
    contours, hierarchy = cv.findContours(labelss, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
    
    # Get the detections as the centroids of the connected components
    detections = []
    n_pix=[]
    
    for i in range(1, num_labels):
        mask = (labels == i)
        count = np.count_nonzero(mask)
        n_pix.append(count)
        cx, cy = centroids[i]
        detections.append((cx, cy))


    # Match the detections to the existing objects using the Hungarian algorithm
    cost_matrix = np.zeros((len(objects), len(detections)))
    for i in range(len(objects)):
        for j in range(len(detections)):
            # Compute the Euclidean distance between the predicted location and the detection
            x = objects[i]
            y = detections[j]
            d = np.sqrt((x[0]-y[0])**2 + (x[1]-y[1])**2)

            # Set the cost as the Euclidean distance
            cost_matrix[i, j] = d

    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    for i, j in zip(row_ind, col_ind):
        # Update the location of the object
        objects[i] = detections[j]
        ll[i]=n_pix[j]

    
    # Add new objects for unmatched detections
    for j in range(len(detections)):
        if j not in col_ind:
            objects.append(detections[j])
            ll.append(n_pix[j])
        
    frame2=frame.copy()
    text = 'Frame:' + str(f)+' ,current objects: '+str(len(detections))+' ,all objects: '+str(len(objects))
    cv.putText(frame2,text , (100, 100), cv.FONT_HERSHEY_COMPLEX, 1, (0,250,0), 1, cv.LINE_AA)

    # Draw the objects on the frame
    for obj in range(len(objects)):
        x, y = int(objects[obj][0]), int(objects[obj][1])
        if x>3:
            tt=str(obj)+' : '+str(x)+','+str(y)
            cv.putText(frame2,tt , (100, 120+20*obj), cv.FONT_HERSHEY_COMPLEX, 0.5, (255,0,0), 1, cv.LINE_AA)
            if ll[obj]>5000 and ll[obj]<7000:
                cv.rectangle(frame2, (x-50, y-50), (x+50, y+50), (0, 255, 0), 2)
            elif ll[obj]>=7000:
                cv.rectangle(frame2, (x-50, y-50), (x+50, y+50), (0, 0, 255), 2)
            else:
                cv.rectangle(frame2, (x-50, y-50), (x+50, y+50), (255, 0, 0), 2)
    frame_array.append(frame2)
    cv.imshow('Conveyor', frame2)

    if cv.waitKey(1) == ord('q'):  
        break

cap.release()
cv.destroyAllWindows()



Writing the video

In [ ]:

# Writing the video
shape = (1080, 1920, 3)


out = cv.VideoWriter('./conveyor_result.mp4',cv.VideoWriter_fourcc(*'h264'), 30, (shape[1], shape[0]))
 
for i in range(len(frame_array)):
    cv.imshow('Frame', frame_array[i])
    if cv.waitKey(1) == ord('q'):
        break
    out.write(frame_array[i])

out.release()
cv.destroyAllWindows()